In [85]:
import pandas as pd
import zipfile
import os
import glob

from src.app.speckle_io import ags3_to_dfs


csv_path = (
    "/Users/jamesholcombe/Documents/git/speckle-hackathon/RelevantReportNo (1).csv"
)

folder = "/Users/jamesholcombe/Documents/git/speckle-hackathon/data/drive-download-20240607T142246Z-001/GI_AGS"

df = pd.read_csv(csv_path)
report_nums = set(df["REPORT_NO"].tolist())
report_nums = set([int(x) for x in report_nums])
print("Number of report numbers: ", len(report_nums))


# for file in os.listdir(folder):
#     if file.endswith(".zip") and int(file.replace(".zip", "")) in report_nums:
#         with zipfile.ZipFile(os.path.join(folder, file), "r") as zip_ref:
#             zip_ref.extractall(folder)
#             print(f"Extracted {file}")
#             # get list of files in the zip
#             files = zip_ref.namelist()
#             print(files)

# TODO: Extract the AGS files that are in the list of report numbers
# create 3 points for each borehole
# upload to speckle

ags_file_paths = []


for report_name in os.listdir(folder):
    if ".zip" in report_name:
        continue

    if int(report_name) in report_nums:
        # print(f"Extracting {report_name}")
        search_path = os.path.join(folder, report_name)
        for f in glob.glob(os.path.join(folder, report_name, "*.ags")):

            ags_file_paths.append(f)

data_strings = []
for file_path in ags_file_paths:
    with open(file_path, "r") as f:
        try:
            data_strings.append(f.read())
        except Exception as e:
            print(f"Could not read {file_path}")


data_dfs = [ags3_to_dfs(data_string) for data_string in data_strings]
data_dfs = [data_df for data_df in data_dfs if "WETH" in data_df.keys()]

df_holes = pd.concat([data_df["HOLE"] for data_df in data_dfs], ignore_index=True)
df_weth = pd.concat([data_df["WETH"] for data_df in data_dfs], ignore_index=True)
common_keys = set.intersection(*[set(data_df.keys()) for data_df in data_dfs])


print("Number of AGS files extracted: ", len(data_dfs))


Number of report numbers:  97
Could not read /Users/jamesholcombe/Documents/git/speckle-hackathon/data/drive-download-20240607T142246Z-001/GI_AGS/70803/GE2021.06.16.ags
Could not read /Users/jamesholcombe/Documents/git/speckle-hackathon/data/drive-download-20240607T142246Z-001/GI_AGS/53815/GE200903.38.ags
Could not read /Users/jamesholcombe/Documents/git/speckle-hackathon/data/drive-download-20240607T142246Z-001/GI_AGS/70921/WO.10 AGS.ags
Could not read /Users/jamesholcombe/Documents/git/speckle-hackathon/data/drive-download-20240607T142246Z-001/GI_AGS/52674/GE200903.26.ags
Could not read /Users/jamesholcombe/Documents/git/speckle-hackathon/data/drive-download-20240607T142246Z-001/GI_AGS/53816/GE200903.38.ags
No data was found on line 4. Last Group: PROJ
No data was found on line 43. Last Group: HOLE
No data was found on line 54. Last Group: HDIA
No data was found on line 121. Last Group: DREM
No data was found on line 259. Last Group: PTIM
No data was found on line 1074. Last Group: S

In [86]:
df_weth.sort_values(["HOLE_ID", "WETH_TOP"], inplace=True)
df_weth["WETH_TOP"] = df_weth["WETH_TOP"].astype(float)




df_weth.loc[df_weth["WETH_GRAD"].isin(["IV", "V", "VI"]), "SOIL_TYPE"] = "Weathered Granite"
df_weth.loc[df_weth["WETH_GRAD"].isin(["I", "II", "III"]), "SOIL_TYPE"] = "Granite"

top_depth_granite = df_weth.loc[df_weth["SOIL_TYPE"] == "Granite"].groupby("HOLE_ID")[
    "WETH_TOP"
].min().rename("MAX_DEPTH_GRANITE").reset_index()

top_depth_weathered = df_weth.loc[df_weth["SOIL_TYPE"] == "Weathered Granite"].groupby("HOLE_ID")[
    "WETH_TOP"
].min().rename("MAX_DEPTH_WEATHERED").reset_index()


df = pd.merge(df_holes, top_depth_granite, on="HOLE_ID", how="right")
df = pd.merge(df, top_depth_weathered, on="HOLE_ID", how="right")
df.dropna(subset=["MAX_DEPTH_GRANITE", "MAX_DEPTH_WEATHERED"], inplace=True)
df["MAX_DEPTH_GRANITE"] = df["MAX_DEPTH_GRANITE"].astype(float)
df["MAX_DEPTH_WEATHERED"] = df["MAX_DEPTH_WEATHERED"].astype(float)
df["HOLE_GL"] = df["HOLE_GL"].astype(float)
df["MAX_DEPTH_GRANITE"] = df["HOLE_GL"] - df["MAX_DEPTH_GRANITE"]
df["MAX_DEPTH_WEATHERED"] = df["HOLE_GL"] - df["MAX_DEPTH_WEATHERED"]


# df_weth.groupby("HOLE_ID")["WETH_TOP"]

In [87]:
df_speckle = df[["HOLE_ID", "HOLE_NATE", "HOLE_NATN", "MAX_DEPTH_GRANITE", "MAX_DEPTH_WEATHERED", "HOLE_GL"]]
df_speckle

,HOLE_ID,HOLE_NATE,HOLE_NATN,MAX_DEPTH_GRANITE,MAX_DEPTH_WEATHERED,HOLE_GL
0,01273/S01,838738.13,820292.42,-47.66,-20.83,-2.83
1,01273/S03,838760.21,820228.73,-49.15,-21.59,-2.09
2,01273/S04,838823.32,820225.23,-47.84,-21.74,-5.24
4,01273/S09,838812.22,820162.28,-48.45,-25.20,-4.70
6,01273/S12,838744.63,820110.20,-70.12,-30.74,-1.24
...,...,...,...,...,...,...
514,TKW-AD5,838026.63,820328.57,-54.84,-25.69,3.91
515,TKW-AD6,838051.79,820315.32,-60.16,-28.05,3.95
516,TKW-AD7,838007.05,820315.81,-56.03,-23.85,3.95
517,TKW-AD8,838025.28,820306.89,-57.77,-25.46,4.04


In [88]:
from typing import Dict, List
import pandas as pd
import datetime as dt
import os
import re

from dotenv import load_dotenv
import pandas as pd

from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account
from specklepy.objects import Base
from specklepy.objects.geometry import Point, Line
from specklepy.transports.server import ServerTransport
from specklepy.api import operations

In [89]:
speckle_server = "app.speckle.systems"
speckle_token = os.environ["TOKEN"]
client = SpeckleClient(host=speckle_server)
client.authenticate_with_token(speckle_token)
stream_id = "013aaf06e7"
transport = ServerTransport(client=client, stream_id=stream_id)

In [90]:
df_speckle.dropna(subset=["HOLE_NATE", "HOLE_NATN", "HOLE_GL"], inplace=True)

/var/folders/rj/3bkhfqz57jd81wj5mln8hw4r0000gn/T/ipykernel_60431/306926114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_speckle.dropna(subset=["HOLE_NATE", "HOLE_NATN", "HOLE_GL"], inplace=True)


In [91]:
obj = Base()

df_speckle["HOLE_ID"] = df_speckle["HOLE_ID"].str.replace("/", "-")
df_speckle["HOLE_GL"] = df_speckle["HOLE_GL"].astype(float)


obj["interfaces"] = {}

for i, row in df_speckle.iterrows():
    
    hole = {}
    sedi_point = Point(
        x=row["HOLE_NATE"],
        y=row["HOLE_NATN"],
        z=row["HOLE_GL"],
    )
    hole["groundLevel"] = sedi_point
    
    
    
    hole["topGranite"] = Point(
        x=row["HOLE_NATE"],
        y=row["HOLE_NATN"],
        z=row["MAX_DEPTH_GRANITE"],
    )
    hole["topWeatheredGranite"] = Point(
        x=row["HOLE_NATE"],
        y=row["HOLE_NATN"],
        z=row["MAX_DEPTH_WEATHERED"],
    )
        
    
    
   
    obj["interfaces"][row["HOLE_ID"]] = hole
    
    
    
   
    
hash = operations.send(base=obj, transports=[transport])
client.commit.create(
        branch_name="geo/gi/borehole/lith-interfaces",
        stream_id=stream_id,
        object_id=hash,
        message=f"these are lines I made in speckle-py at {dt.datetime.now()}",
    )


/var/folders/rj/3bkhfqz57jd81wj5mln8hw4r0000gn/T/ipykernel_60431/3553131982.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_speckle["HOLE_ID"] = df_speckle["HOLE_ID"].str.replace("/", "-")
/var/folders/rj/3bkhfqz57jd81wj5mln8hw4r0000gn/T/ipykernel_60431/3553131982.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_speckle["HOLE_GL"] = df_speckle["HOLE_GL"].astype(float)


'1587d0f651'